In [1]:
from elasticsearch import Elasticsearch
#es_client = Elasticsearch('http://localhost:9200')
es_client = Elasticsearch('http://host.docker.internal:9200')

In [2]:
!curl http://host.docker.internal:9200

{
  "name" : "c84352211afb",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "Ga49TmrdSu2eTfz95bYysg",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [9]:
import requests

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [6]:

index_name = "course-questions"

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}


es_client.indices.create(index=index_name, body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/BkGnUJZuSrqRGSS745w_Bw] already exists')

In [5]:
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [7]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },

            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    print("RESP: ", response)
    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [8]:
query_question = 'How do execute a command on a Kubernetes pod?'

In [9]:
search_results = elastic_search(query_question)
print(search_results)

RESP:  {'took': 26, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 739, 'relation': 'eq'}, 'max_score': 44.50556, 'hits': [{'_index': 'course-questions', '_id': 'jPIHdpcBsF65hyciCAF9', '_score': 44.50556, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'G_IHdpcBsF65hyciDAI9', '_score': 35.433445, '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke', 'section': '10. Kubernetes and TensorFlow Serving', 'question': 'Kubernetes-dashbo

In [10]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [11]:
query_question = "How do copy a file to a Docker container?"
search_results = elastic_search(query_question)
print(search_results[2]['question'])

How do I copy files from a different folder into docker container’s working directory?


In [12]:
context_template = ''

for r in search_results:
    question = r['question']
    text = r['text']

    context_template = context_template + \
    f"""
Q: {question}
A: {text}
    """.strip() + "\n\n"


In [13]:
print(context_template)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [14]:
prompt_template = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {query_question}

CONTEXT:
{context_template}
""".strip()

In [15]:
print(prompt_template)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do copy a file to a Docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/contain

In [16]:
print(len(context_template))

1220


In [23]:
import boto3
import json

import boto3.session

session = boto3.session.Session(profile_name="dfxuser", region_name="us-west-2")

bedrock_runtime_client = session.client('bedrock-runtime')
#%nv AWS_PROFILE=jose
#%env AWS_REGION=us-west-2


In [24]:
client = session.client("sts")
print(client.get_caller_identity()["Account"])

087045461309


In [25]:
MICRO_MODEL_ID = "us.amazon.nova-micro-v1:0"
LITE_MODEL_ID = "us.amazon.nova-lite-v1:0"
PRO_MODEL_ID = "us.amazon.nova-pro-v1:0"

In [26]:
def call_nova(model,
              messages,
              system_message='',
              stream='off',
              max_tokens=1024,
              temp=0.7,
              top_p=0.99,
              top_k=20,
              tools=None,
              verbose=False):
    system_list = [{"text": system_message}]
    inf_params = {"max_new_tokens": max_tokens, "top_p": top_p, "top_k": top_k,
                  "temperature": temp}
    request_body = {"messages": messages,
                    "system": system_list,
                    "inferenceConfig": inf_params
                   }
    if tools is not None:
        tool_config=[]
        for tool in tools:
            tool_config.append({"toolSpec": tool})
        request_body['toolConfig'] = {"tools":tool_config}
    if verbose:
        print("Request Body", request_body)
    if stream=='off':
        response = bedrock_runtime_client.invoke_model(modelId=model, body=json.dumps(request_body))
        model_response = json.loads(response["body"].read())
        return model_response, model_response["output"]["message"]["content"][0]['text']
    else:
        response = bedrock_runtime_client.invoke_model_with_response_stream(modelId=model, body=json.dumps(request_body))
        return response["body"]

In [27]:
messages = [{"role": "user", "content": [{"text": prompt_template}]}]
system_message = ""
model_response, content_text = call_nova(
    model=MICRO_MODEL_ID,
    messages=messages,
    system_message=system_message
    )

print("\n[Full Response]")
print(json.dumps(model_response, indent=2))
print("\n[Response Content Text]")
print(content_text)


[Full Response]
{
  "output": {
    "message": {
      "content": [
        {
          "text": "To copy a file to a Docker container, you can use the `docker cp` command. Here's a step-by-step guide based on the provided context:\n\n1. **Identify the Container ID**: First, you need to find the container ID of the running container where you want to copy the file. You can list all running containers using the following command:\n   ```sh\n   docker ps\n   ```\n   This command will display a list of all running containers, and you can identify the container ID from the output.\n\n2. **Use the `docker cp` Command**: Once you have the container ID, you can use the `docker cp` command to copy the file from your local machine to the container. The basic syntax is:\n   ```sh\n   docker cp /path/to/local/file_or_directory container_id:/path/in/container\n   ```\n   For example, if you want to copy a file named `example.txt` from your local machine to the working directory of a container with

ClientError: An error occurred (InvalidClientTokenId) when calling the GetCallerIdentity operation: The security token included in the request is invalid.